In [19]:
import pandas as pd 
path = "../../data/introduction/introduction.txt"

with open(path, "r", encoding="utf8") as f:
    lines = f.readlines()
lines

['This website presents the results of our publication “Analyzing the multidimensionality of biological aging with the tools of deep learning across diverse image-based and physiological indicators yields robust age predictors” which can be found at www.biorxiv/XXTODO.\n',
 '\n',
 'We analyzed 676,787 samples from 502,211 UK Biobank participants aged 37-82 years with the tools of deep learning to build a total of 331 chronological age predictors on different data modalities such as videos (e.g heart magnetic resonance imaging [MRI] videos), images (e.g brain, liver and pancreas MRIs, full body X-rays, eye fundus and optical coherence tomography [OCT] images, carotid ultrasound images), time-series (e.g electrocardiograms [ECGs], pulse wave analysis, wrist accelerometer data) and scalar data (e.g blood biomarkers, anthropometric measures) to characterize the multiple dimensions of aging. We combined these age predictors into 28 ensemble models based on specific organ systems. We then de